# パターン表示回路プログラム
- PLに実装したパターン表示回路の出力をDisplayPortに出力（ライブビデオ出力）する

### [STEP-1] レジスタアドレス定義
- 変更に必要なレジスタのみを定義

In [ ]:
# ZYNQ内部レジスタ
DP_REG_BASE_ADDR  = 0xfd4a0000
DP_REG_ADDR_RANGE = 0x0000cc20

RA_DP_MAIN_STREAM_ENABLE            = 0x00000084
RA_V_BLEND_SET_GLOVAL_ALPHA_REG     = 0x0000a00c
RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT = 0x0000b070
RA_AV_BUF_AUD_VID_CLK_SOURCE        = 0x0000b120
RA_AV_BUF_SRST_REG                  = 0x0000b124

#GPIOレジスタ
RA_GPIO_DATA = 0x00000000

### [STEP-2] Overlay
- パターン表示回路のコンフィギュレーションを実施

In [ ]:
from pynq import Overlay
pl = Overlay("design_1.bit")

### [STEP-3] MMIO初期化
- ZYNQ内部レジスタアクセス用のオブジェクトdpregを生成

In [ ]:
from pynq import MMIO
dpreg = MMIO(DP_REG_BASE_ADDR,DP_REG_ADDR_RANGE)

### [STEP-4] パターン表示回路の解像度設定
- RESOLの値を変更してパターン表示回路の解像度を設定
    - 0 : VGA設定
    - 1 : XGA設定
    - 2 : SXGA設定
-「PYNQシステム環境構築体験」では、RESOLの値は0,1のみ対応

In [ ]:
RESOL = 0
pl.axi_gpio_0.write(RA_GPIO_DATA,(RESOL&1))
pl.axi_gpio_0.read(RA_GPIO_DATA)

### [STEP-5] DisplayPort初期化
- パターン表示回路の解像度に設定を合わせる
    - VGA設定  : dp.configure(VideoMode(640,480,24),PIXEL_RGB)
    - XGA設定  : dp.configure(VideoMode(1024,768,24),PIXEL_RGB)
    - SXGA設定 : dp.configure(VideoMode(1280,1024,24),PIXEL_RGB)

In [ ]:
from pynq.lib.video import *
dp = DisplayPort()

if (RESOL & 1) == 1 :
    print("XGA setting @ DisplayPort")
    dp.configure(VideoMode(1024,768,24),PIXEL_RGB)    #XGA
else :
    print("VGA setting @ DisplayPort")
    dp.configure(VideoMode(640,480,24),PIXEL_RGB)     #VGA

### [STEP-6] DisplayPortレジスタ設定
- dpregのメソッドを使ってZynq内部のDisplayPortレジスタにアクセス
- 表示クロック（DCLK)および表示タイミング制御信号（VSYNC, HSYNC, DE)をPLから供給するように設定

In [ ]:
import time
dpreg.write(RA_DP_MAIN_STREAM_ENABLE,0x00000000)
dpreg.write(RA_V_BLEND_SET_GLOVAL_ALPHA_REG,0x00000000)
dpreg.write(RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT,0x00000050)
dpreg.write(RA_AV_BUF_AUD_VID_CLK_SOURCE,0x00000002)
dpreg.write(RA_AV_BUF_SRST_REG,0x00000002)
time.sleep(1)
dpreg.write(RA_AV_BUF_SRST_REG,0x00000000)
time.sleep(1)
dpreg.write(RA_DP_MAIN_STREAM_ENABLE,0x00000001)

### [STEP-7] PYNQの初期画面に戻す
- DisplayPortの制御は、PL側からPS側に切り替わります

In [ ]:
dp.close()
dpreg.write(RA_DP_MAIN_STREAM_ENABLE,0x00000000)